## Getting a random + hillclimber fixed using imports
The goal of this notebook is to get the whole pipeline fixed using as less as code as possible by encapsulating most of the code as reusable code in external .py files

In [1]:
import geopandas as gpd
import shapely
from getpass import getpass

import sys
sys.path.append("../Code")

from helper_functions import *
from loading_data import *

WelkomCorne!


In [2]:
# polygons_list = load_geodata_containers()

In [3]:
api_df = load_api_data(prnt=False)

In [4]:
rel_poi_df = get_db_afvalcluster_info()

NameError: name 'get_db_afvalcluster_info' is not defined

In [ ]:
joined = join_api_db(rel_poi_df, api_df)